# Using Azure Open Datasets in Synapse - Enrich NYC Green Taxi Data with Holiday and Weather

Synapse has [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/) package pre-installed. This notebook provides examples of how to enrich NYC Green Taxi Data with Holiday and Weather with focusing on :
- read Azure Open Dataset
- manipulate the data to prepare for further analysis, including column projection, filtering, grouping and joins etc. 
- create a Spark table to be used in other notebooks for modeling training

## Data loading 
Let's first load the [NYC green taxi trip records](https://azure.microsoft.com/en-us/services/open-datasets/catalog/nyc-taxi-limousine-commission-green-taxi-trip-records/). The Open Datasets package contains a class representing each data source (NycTlcGreen for example) to easily filter date parameters before downloading.

In [1]:
from azureml.opendatasets import NycTlcGreen

from datetime import datetime
from dateutil import parser
end_date = parser.parse('2018-06-06')
start_date = parser.parse('2018-05-01')

nyc_tlc = NycTlcGreen(start_date=start_date, end_date=end_date)
nyc_tlc_df = nyc_tlc.to_spark_dataframe()

StatementMeta(small, 12, 3, Finished, Available)

In [2]:
# Display 5 rows

nyc_tlc_df.show(5, truncate = False)

StatementMeta(small, 12, 4, Finished, Available)

+--------+-------------------+-------------------+--------------+------------+------------+------------+---------------+--------------+----------------+---------------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+--------+-----------+--------+------+-------+
|vendorID|lpepPickupDatetime |lpepDropoffDatetime|passengerCount|tripDistance|puLocationId|doLocationId|pickupLongitude|pickupLatitude|dropoffLongitude|dropoffLatitude|rateCodeID|storeAndFwdFlag|paymentType|fareAmount|extra|mtaTax|improvementSurcharge|tipAmount|tollsAmount|ehailFee|totalAmount|tripType|puYear|puMonth|
+--------+-------------------+-------------------+--------------+------------+------------+------------+---------------+--------------+----------------+---------------+----------+---------------+-----------+----------+-----+------+--------------------+---------+-----------+--------+-----------+--------+------+-------+
|2       |2018-06-02 14:10:02|2018-06-02

Now that the initial data is loaded. Let's do some projection on the data to 
- create new columns for the month number, day of month, day of week, and hour of day. These info is going to be used in the training model to factor in time-based seasonality.
- add a static feature for the country code to join holiday data. 

In [3]:
# Extract month, day of month, and day of week from pickup datetime and add a static column for the country code to join holiday data. 

import pyspark.sql.functions as f

nyc_tlc_df_expand = nyc_tlc_df.withColumn('datetime',f.to_date('lpepPickupDatetime'))\
                .withColumn('month_num',f.month(nyc_tlc_df.lpepPickupDatetime))\
                .withColumn('day_of_month',f.dayofmonth(nyc_tlc_df.lpepPickupDatetime))\
                .withColumn('day_of_week',f.dayofweek(nyc_tlc_df.lpepPickupDatetime))\
                .withColumn('hour_of_day',f.hour(nyc_tlc_df.lpepPickupDatetime))\
                .withColumn('country_code',f.lit('US'))

StatementMeta(small, 12, 5, Finished, Available)

Remove some of the columns that won't need for modeling or additional feature building.




In [4]:
# Remove unused columns from nyc green taxi data

columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "pickupLongitude", 
                     "pickupLatitude", "dropoffLongitude","dropoffLatitude" ,"rateCodeID", 
                     "storeAndFwdFlag","paymentType", "fareAmount", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType "  
                    ]

nyc_tlc_df_clean = nyc_tlc_df_expand.select([column for column in nyc_tlc_df_expand.columns if column not in columns_to_remove])

StatementMeta(small, 12, 6, Finished, Available)

In [5]:
# Display 5 rows
nyc_tlc_df_clean.show(5)

StatementMeta(small, 12, 7, Finished, Available)

+--------+-------------------+--------------+------------+---------+-----------+--------+------+-------+----------+---------+------------+-----------+-----------+------------+
|vendorID| lpepPickupDatetime|passengerCount|tripDistance|tipAmount|totalAmount|tripType|puYear|puMonth|  datetime|month_num|day_of_month|day_of_week|hour_of_day|country_code|
+--------+-------------------+--------------+------------+---------+-----------+--------+------+-------+----------+---------+------------+-----------+-----------+------------+
|       2|2018-06-02 14:10:02|             1|         2.5|     3.06|      18.36|       1|  2018|      6|2018-06-02|        6|           2|          7|         14|          US|
|       2|2018-06-02 14:36:36|             1|        0.45|      0.0|        5.8|       1|  2018|      6|2018-06-02|        6|           2|          7|         14|          US|
|       2|2018-06-04 11:18:01|             1|         0.8|      0.0|        5.3|       1|  2018|      6|2018-06-04|     

## Enrich with holiday data
Now that we have taxi data downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. 

Let's load the [public holidays](https://azure.microsoft.com/en-us/services/open-datasets/catalog/public-holidays/) from Azure Open datasets.


In [6]:
from azureml.opendatasets import PublicHolidays

hol = PublicHolidays(start_date=start_date, end_date=end_date)
hol_df = hol.to_spark_dataframe()

# Display data
hol_df.show(5, truncate = False)

StatementMeta(small, 12, 8, Finished, Available)

+---------------+----------------------------+----------------------------+-------------+-----------------+-------------------+
|countryOrRegion|holidayName                 |normalizeHolidayName        |isPaidTimeOff|countryRegionCode|date               |
+---------------+----------------------------+----------------------------+-------------+-----------------+-------------------+
|Argentina      |Día del Trabajo [Labour Day]|Día del Trabajo [Labour Day]|null         |AR               |2018-05-01 00:00:00|
|Austria        |Staatsfeiertag              |Staatsfeiertag              |null         |AT               |2018-05-01 00:00:00|
|Belarus        |Праздник труда              |Праздник труда              |null         |BY               |2018-05-01 00:00:00|
|Belgium        |Dag van de Arbeid           |Dag van de Arbeid           |null         |BE               |2018-05-01 00:00:00|
|Brazil         |Dia Mundial do Trabalho     |Dia Mundial do Trabalho     |null         |BR             

Rename the countryRegionCode and date columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. 

In [7]:
hol_df_clean = hol_df.withColumnRenamed('countryRegionCode','country_code')\
            .withColumn('datetime',f.to_date('date'))

hol_df_clean.show(5)

StatementMeta(small, 12, 9, Finished, Available)

+---------------+--------------------+--------------------+-------------+------------+-------------------+----------+
|countryOrRegion|         holidayName|normalizeHolidayName|isPaidTimeOff|country_code|               date|  datetime|
+---------------+--------------------+--------------------+-------------+------------+-------------------+----------+
|      Argentina|Día del Trabajo [...|Día del Trabajo [...|         null|          AR|2018-05-01 00:00:00|2018-05-01|
|        Austria|      Staatsfeiertag|      Staatsfeiertag|         null|          AT|2018-05-01 00:00:00|2018-05-01|
|        Belarus|      Праздник труда|      Праздник труда|         null|          BY|2018-05-01 00:00:00|2018-05-01|
|        Belgium|   Dag van de Arbeid|   Dag van de Arbeid|         null|          BE|2018-05-01 00:00:00|2018-05-01|
|         Brazil|Dia Mundial do Tr...|Dia Mundial do Tr...|         null|          BR|2018-05-01 00:00:00|2018-05-01|
+---------------+--------------------+------------------

Next, join the holiday data with the taxi data by performing a left-join. This will preserve all records from taxi data, but add in holiday data where it exists for the corresponding datetime and country_code, which in this case is always "US". Preview the data to verify that they were merged correctly.

In [8]:
# enrich taxi data with holiday data
nyc_taxi_holiday_df = nyc_tlc_df_clean.join(hol_df_clean, on = ['datetime', 'country_code'] , how = 'left')

nyc_taxi_holiday_df.show(5)

StatementMeta(small, 12, 10, Finished, Available)

+----------+------------+--------+-------------------+--------------+------------+---------+-----------+--------+------+-------+---------+------------+-----------+-----------+---------------+-----------+--------------------+-------------+----+
|  datetime|country_code|vendorID| lpepPickupDatetime|passengerCount|tripDistance|tipAmount|totalAmount|tripType|puYear|puMonth|month_num|day_of_month|day_of_week|hour_of_day|countryOrRegion|holidayName|normalizeHolidayName|isPaidTimeOff|date|
+----------+------------+--------+-------------------+--------------+------------+---------+-----------+--------+------+-------+---------+------------+-----------+-----------+---------------+-----------+--------------------+-------------+----+
|2018-06-02|          US|       2|2018-06-02 14:10:02|             1|         2.5|     3.06|      18.36|       1|  2018|      6|        6|           2|          7|         14|           null|       null|                null|         null|null|
|2018-06-02|          US

In [9]:
# Create a temp table and filter out non empty holiday rows

nyc_taxi_holiday_df.createOrReplaceTempView("nyc_taxi_holiday_df")
spark.sql("SELECT * from nyc_taxi_holiday_df WHERE holidayName is NOT NULL ").show(5, truncate = False)

StatementMeta(small, 12, 11, Finished, Available)

+----------+------------+--------+-------------------+--------------+------------+---------+-----------+--------+------+-------+---------+------------+-----------+-----------+---------------+------------+--------------------+-------------+-------------------+
|datetime  |country_code|vendorID|lpepPickupDatetime |passengerCount|tripDistance|tipAmount|totalAmount|tripType|puYear|puMonth|month_num|day_of_month|day_of_week|hour_of_day|countryOrRegion|holidayName |normalizeHolidayName|isPaidTimeOff|date               |
+----------+------------+--------+-------------------+--------------+------------+---------+-----------+--------+------+-------+---------+------------+-----------+-----------+---------------+------------+--------------------+-------------+-------------------+
|2018-05-28|US          |2       |2018-05-28 10:28:09|1             |2.01        |2.26     |13.56      |1       |2018  |5      |5        |28          |2          |10         |United States  |Memorial Day|Memorial Day    

## Enrich with weather data¶

Now we append NOAA surface weather data to the taxi and holiday data. Use a similar approach to fetch the [NOAA weather history data](https://azure.microsoft.com/en-us/services/open-datasets/catalog/noaa-integrated-surface-data/) from Azure Open Datasets. 

In [10]:
from azureml.opendatasets import NoaaIsdWeather

isd = NoaaIsdWeather(start_date, end_date)
isd_df = isd.to_spark_dataframe()

StatementMeta(small, 12, 12, Finished, Available)

In [11]:
isd_df.show(5, truncate = False)

StatementMeta(small, 12, 13, Finished, Available)

+------+-----+-------------------+--------+---------+---------+---------+---------+-----------+--------------+-------------+-----------------------+--------------------+----------+-----------+---------+-------------------------+---------------+------------+----+---+-------+-----+
|usaf  |wban |datetime           |latitude|longitude|elevation|windAngle|windSpeed|temperature|seaLvlPressure|cloudCoverage|presentWeatherIndicator|pastWeatherIndicator|precipTime|precipDepth|snowDepth|stationName              |countryOrRegion|p_k         |year|day|version|month|
+------+-----+-------------------+--------+---------+---------+---------+---------+-----------+--------------+-------------+-----------------------+--------------------+----------+-----------+---------+-------------------------+---------------+------------+----+---+-------+-----+
|720602|00194|2018-05-08 12:35:00|33.25   |-81.383  |75.0     |40       |4.1      |18.0       |null          |null         |null                   |null     

In [12]:
# Filter out weather info for new york city, remove the recording with null temperature 

weather_df = isd_df.filter(isd_df.latitude >= '40.53')\
                        .filter(isd_df.latitude <= '40.88')\
                        .filter(isd_df.longitude >= '-74.09')\
                        .filter(isd_df.longitude <= '-73.72')\
                        .filter(isd_df.temperature.isNotNull())\
                        .withColumnRenamed('datetime','datetime_full')
                         

StatementMeta(small, 12, 14, Finished, Available)

In [13]:
# Remove unused columns

columns_to_remove_weather = ["usaf", "wban", "longitude", "latitude"]
weather_df_clean = weather_df.select([column for column in weather_df.columns if column not in columns_to_remove_weather])\
                        .withColumn('datetime',f.to_date('datetime_full'))

weather_df_clean.show(5, truncate = False)

StatementMeta(small, 12, 15, Finished, Available)

+-------------------+---------+---------+---------+-----------+--------------+-------------+-----------------------+--------------------+----------+-----------+---------+------------------------------------+---------------+------------+----+---+-------+-----+----------+
|datetime_full      |elevation|windAngle|windSpeed|temperature|seaLvlPressure|cloudCoverage|presentWeatherIndicator|pastWeatherIndicator|precipTime|precipDepth|snowDepth|stationName                         |countryOrRegion|p_k         |year|day|version|month|datetime  |
+-------------------+---------+---------+---------+-----------+--------------+-------------+-----------------------+--------------------+----------+-----------+---------+------------------------------------+---------------+------------+----+---+-------+-----+----------+
|2018-05-20 15:00:00|7.0      |230      |8.2      |23.9       |1014.3        |BKN          |null                   |null                |null      |null       |null     |JOHN F KENNEDY IN

Next group the weather data so that you have daily aggregated weather values. 


In [14]:
# Enrich weather data with aggregation statistics

aggregations = {"snowDepth": "mean", "precipTime": "max", "temperature": "mean", "precipDepth": "max"}
weather_df_grouped = weather_df_clean.groupby("datetime").agg(aggregations)

StatementMeta(small, 12, 16, Finished, Available)

In [15]:
weather_df_grouped.show(5)

StatementMeta(small, 12, 17, Finished, Available)

+----------+--------------+------------------+---------------+----------------+
|  datetime|avg(snowDepth)|  avg(temperature)|max(precipTime)|max(precipDepth)|
+----------+--------------+------------------+---------------+----------------+
|2018-05-28|          null| 15.33363636363636|           24.0|          2540.0|
|2018-06-06|          null|              21.4|            6.0|             0.0|
|2018-05-26|          null|26.072330097087377|           24.0|          2540.0|
|2018-05-27|          null|18.931365313653142|           24.0|          7648.0|
|2018-06-03|          null|18.242803030303037|           24.0|          2540.0|
+----------+--------------+------------------+---------------+----------------+
only showing top 5 rows

In [16]:
# Rename columns

weather_df_grouped = weather_df_grouped.withColumnRenamed('avg(snowDepth)','avg_snowDepth')\
                                       .withColumnRenamed('avg(temperature)','avg_temperature')\
                                       .withColumnRenamed('max(precipTime)','max_precipTime')\
                                       .withColumnRenamed('max(precipDepth)','max_precipDepth')

StatementMeta(small, 12, 18, Finished, Available)

Merge the taxi and holiday data you prepared with the new weather data. This time you only need the datetime key, and again perform a left-join of the data. Run the describe() function on the new dataframe to see summary statistics for each field.

In [17]:
# enrich taxi data with weather
nyc_taxi_holiday_weather_df = nyc_taxi_holiday_df.join(weather_df_grouped, on = 'datetime' , how = 'left')
nyc_taxi_holiday_weather_df.cache()

StatementMeta(small, 12, 19, Finished, Available)

DataFrame[datetime: date, country_code: string, vendorID: int, lpepPickupDatetime: timestamp, passengerCount: int, tripDistance: double, tipAmount: double, totalAmount: double, tripType: int, puYear: int, puMonth: int, month_num: int, day_of_month: int, day_of_week: int, hour_of_day: int, countryOrRegion: string, holidayName: string, normalizeHolidayName: string, isPaidTimeOff: boolean, date: timestamp, avg_snowDepth: double, avg_temperature: double, max_precipTime: double, max_precipDepth: double]

In [18]:
nyc_taxi_holiday_weather_df.show(5)

StatementMeta(small, 12, 20, Finished, Available)

+----------+------------+--------+-------------------+--------------+------------+---------+-----------+--------+------+-------+---------+------------+-----------+-----------+---------------+------------+--------------------+-------------+-------------------+-------------+-----------------+--------------+---------------+
|  datetime|country_code|vendorID| lpepPickupDatetime|passengerCount|tripDistance|tipAmount|totalAmount|tripType|puYear|puMonth|month_num|day_of_month|day_of_week|hour_of_day|countryOrRegion| holidayName|normalizeHolidayName|isPaidTimeOff|               date|avg_snowDepth|  avg_temperature|max_precipTime|max_precipDepth|
+----------+------------+--------+-------------------+--------------+------------+---------+-----------+--------+------+-------+---------+------------+-----------+-----------+---------------+------------+--------------------+-------------+-------------------+-------------+-----------------+--------------+---------------+
|2018-05-28|          US|      

In [19]:
# Run the describe() function on the new dataframe to see summary statistics for each field.

display(nyc_taxi_holiday_weather_df.describe())

StatementMeta(small, 12, 21, Finished, Available)

SynapseWidget(Synapse.DataFrame, 50b52121-74c2-4b97-933b-557c2cee5a3d)

The summary statistics shows that the totalAmount field has negative values, which don't make sense in the context.



In [20]:
# Remove invalid rows with less than 0 taxi fare or tip
final_df = nyc_taxi_holiday_weather_df.filter(nyc_taxi_holiday_weather_df.tipAmount > 0)\
                                      .filter(nyc_taxi_holiday_weather_df.totalAmount > 0)

StatementMeta(small, 12, 22, Finished, Available)

## Cleaning up the existing Database

First we need to drop the tables since Spark requires that a database is empty before we can drop the Database.

Then we recreate the database and set the default database context to it.

In [21]:
spark.sql("DROP TABLE IF EXISTS NYCTaxi.nyc_taxi_holiday_weather"); 

StatementMeta(small, 12, 23, Finished, Available)

DataFrame[]

In [22]:
spark.sql("DROP DATABASE IF EXISTS NYCTaxi"); 
spark.sql("CREATE DATABASE NYCTaxi"); 
spark.sql("USE NYCTaxi");

StatementMeta(small, 12, 24, Finished, Available)

DataFrame[]

## Creating a new table
We create a nyc_taxi_holiday_weather table from the nyc_taxi_holiday_weather dataframe.


In [23]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

final_df.write.saveAsTable("nyc_taxi_holiday_weather");
spark.sql("SELECT COUNT(*) FROM nyc_taxi_holiday_weather").show();

StatementMeta(small, 12, 25, Finished, Available)

+--------+
|count(1)|
+--------+
|  337444|
+--------+